# **Treinamento de um modelo GPT-2**
Este colab contém recursos necessários para criar e treinar um modelo de aprendizado de máquina usando a tecnologia GPT-2 do OpenAI. Este collab faz uso do `gpt-2-simple`, um fork do GPT-2 que torna as coisas mais simples de serem usadas, e não requer uso da API oficial da OpenAI. Você pode ver o código do gpt-2-simple no [GitHub](https://github.com/minimaxir/gpt-2-simple).

Este colab é feito baseado no criado pelo [Max Woolf](https://minimaxir.com/), e é uma versão simplificada e em português dele. Primariamente, este colab está sendo usado para treinamento da base de dados do [Grand Prognosticator](https://github.com/thaalesalves/grand-prognosticator), um robô para consulta de lore de Elder Scrolls. 

Max Woolf escreveu um artigo sobre como criar aplicações que utilizam o modelo de dados treinado pelo GPT-2, e você pode ler mais [aqui](https://minimaxir.com/2019/09/howto-gpt2/).

Infelizmente, por conta dos recursos que o Google fornece, é apenas possível treinar o modelo _small_ (124M) e o modelo _medium_ (355M). Os demais modelos podem ser treinados em infraestruturas mais potentes caso você tenha os recursos para bancá-las.

## Como este colab funciona
Ao executar as células fornecidas, trechos de código Python serão executados no runtime do Google. Uma VM é criada como ambiente de execução, e você tem recursos limitados. Estes recursos são resetados e zeram a cada 4 horas, então não faça treinos muito longos e lembre-se de salvar o seu progresso ao término de cada sessão de treino.

Quando o tempo limite é atingido e o ambiente redefinido, é necessário remontar o seu drive e copiar novamente o checkpoint utilizado, além dos arquivos de treino. 

## Como usar este colab
1. Faça uma cópia deste notebook para o seu drive
2. Execute todo o processo no Chrome (ou navegadores baseados em Chromium)
3. Execute as células providas abaixo


# **Baixando o TensorFlow e do modelo**
O download do TensorFlow deve ser executado a cada sessão de treino, pois entre o término de um processo de treino e o início do próximo é necessário reiniciar a VM. O download do modelo, no entando, deve ser executado uma vez somente, pois o download ficará no armazenamento virtual até que vençam as 4 horas de execução.

Apenas os modelos 144M e o 355M são capazes de serem treinados, portanto prefira-os. O colab não tem recursos suficientes para treinar modelos maiores. Mas você pode baixá-los para usar no chatbot. As opções são

* **124M:** o modelo "small", 500MB em disco.
* **355M:** o modelo "medium", 1.5GB em disco.
* **774M:** o modelo "large", não pode ser treinado no colab, mas pode ser usado no chatbot.
* **1558M:** o modelo "extra large", não pode ser treinado no colab, mas pode ser usado no chatbot.

In [ ]:
# Definição de variáveis para execução (nomes padrão, podem ser alterados)
gpt_model = "355M" # dar preferência para 355M ou 124M
model_name = "run1" # nome do modelo gerado e treinado

In [ ]:
# Iniciando o TensorFlow (deve ser executado uma vez por sessão)
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import shutil
import os
import sys
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

In [ ]:
# Download do modelo do GPT-2 (deve ser executado sempre que o ambiente for redefinido)
gpt2.download_gpt2(model_name = gpt_model)

# **Utilizando GPU**
Este colab utilizará GPU para treinamento do modelo, pois tem mais performance que CPU. O Colaboratory usa uma NVIDIA T4 ou uma K80 para executar procedimentos, dependendo da que estiver disponível. Execute o trecho de código abaixo para verificar qual GPU estará disponível para treino.

In [ ]:
# Verificando GPU disponível
!nvidia-smi

# **Gerenciamento de armazenamento**
É recomendável não subir arquivos diretamente para o ambiente de execução. Suba os arquivos para o seu drive, e copie-os de lá para cá. Assim, você mantém um backup deles para treinos posteriores. Além disso, você poderá também copiar seu modelo treinado para o drive, e depois copiá-lo de volta para cá e continuar o treino de onde parou.

Seu modelo, ao ser copiado para o driver, será empacotado num arquivo `.tar`. Ao ser copiado de volta para o ambiente, o `.tar` será extraído e colocado na pasta correta.

In [ ]:
# Montar o drive no ambiente
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
# Copiar modelo daqui para o drive
gpt2.copy_checkpoint_to_gdrive(run_name = model_name)

In [ ]:
# Copiar modelo do drive para cá
gpt2.copy_checkpoint_from_gdrive(run_name = model_name)

In [ ]:
# Copiar arquivos do drive para cá
dataset = "highrock.txt" # nome do arquivo de dados a ser copiado do drive
gpt2.copy_file_from_gdrive(dataset)

# **Treinamento do modelo**
Com os diretórios criados, o TensorFlow iniciado e o modelo baixado, podemos dar início no treino do nosso modelo. O trecho de código abaixo inicia uma sessão do GPT-2. Esta sessão será usada para treino e para geração de texto a partir do modelo. 

Para treinar o modelo, você terá duas opções: uma inicia o treino do zero, e a segunda continua o treino de onde a sessão anterior parou. Lembre-se de copiar o modelo treinado entre o ambiente e o seu drive para não perdê-lo.

Após a conclusão de uma sessão de treino, você poderá gerar quantas entradas de texto quiser, mas para treinar mais seu modelo, deverá reinciar a VM e iniciar uma nova sessão. 

Menu superior -> Runtime -> Restart runtime

In [ ]:
# Iniciando uma sessão
sess = gpt2.start_tf_sess()

In [ ]:
# Iniciando um treino do zero
gpt2.finetune(sess,
              dataset = dataset,
              model_name = gpt_model,
              steps = 2000,
              restore_from = 'fresh',
              run_name = model_name,
              print_every = 100,
              sample_every = 500,
              save_every = 500)

In [ ]:
# Continuando o treino
gpt2.finetune(sess,
              dataset = dataset,
              model_name = gpt_model,
              steps = 2000,
              restore_from = 'latest',
              run_name = model_name,
              print_every = 100,
              sample_every = 500,
              save_every = 500,
              overwrite = 'true')

# **Geração de texto**
Após o treinamento do seu modelo, será possível gerar entradas de texto com ele. Você pode gerar textos aleatórios ou a partir de um prompt, que será completado pelo modelo.

Atente-se aos parâmetros passados na geração por prompt. Quanto menor a temperatura, mais conciso o texto. Por consequência, quanto maior a tempertura, mais maluco será o texto gerado. O valor padrão definido é `0.5`, mas é possível alterá-lo. O valor mínimo aceito pelo GPT-2 é `0.1`.

* **length:** tamanho do texto gerado em caracteres
* **temperature:** qualidade do texto gerado. Quanto maior o valor, pior a qualidade
* **prefix:** valor de início, que a IA deverá completar com texto próprio
* **nsamples:** número de textos gerados

In [ ]:
# Geração de texto aleatório
gpt2.generate(sess, run_name = model_name)

In [ ]:
# Geração de texto a partir de prompt
prompt = "Balfiera is"
gpt2.generate(sess,
              length = 200,
              temperature = 0.5,
              prefix = prompt,
              nsamples = 15,
              batch_size = 15)

# **Chat com o modelo (em desenvolvimento)**
Além de gerar textos, podemos usar o modelo de AI como um chatbot e interagir com ele.


In [ ]:
# Declaração de varíaveis
chatbot_model = 'run2'

In [ ]:
# Cópia de modelo treinado para a pasta do chatbot (antes de usar o chatbot)
shutil.move('checkpoint/' + model_name, 'models/')

'models/run2'

In [ ]:
# Cópia de modelo treinado de volta para a pasta original (depois de terminar de usar o chatbot)
shutil.move('models/' + model_name, 'checkpoint/')

'checkpoint/run2'

In [ ]:
# Clone do projeto do GPT
!git clone https://github.com/nshepperd/gpt-2.git

# Importação das libs
sys.path.append("/content/gpt-2/src")

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [ ]:
# Criação do chatbot

!pip install fire

import fire
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder
import generate_unconditional_samples
import interactive_conditional_samples

class GPT2:

  
  # extracted from the source code to generate some text based on a prior
  def __init__(
      self,
      model_name=chatbot_model,
      seed=None,
      nsamples=1,
      batch_size=1,
      length=None,
      temperature=1,
      top_k=0,
      raw_text="",
  ):
      """
      Interactively run the model
      :model_name=117M : String, which model to use
      :seed=None : Integer seed for random number generators, fix seed to reproduce
       results
      :nsamples=1 : Number of samples to return total
      :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
      :length=None : Number of tokens in generated text, if None (default), is
       determined by model hyperparameters
      :temperature=1 : Float value controlling randomness in boltzmann
       distribution. Lower temperature results in less random completions. As the
       temperature approaches zero, the model will become deterministic and
       repetitive. Higher temperature results in more random completions.
      :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
       considered for each step (token), resulting in deterministic completions,
       while 40 means 40 words are considered at each step. 0 (default) is a
       special setting meaning no restrictions. 40 generally is a good value.
      """
      if batch_size is None:
          batch_size = 1
      assert nsamples % batch_size == 0

      self.nsamples = nsamples
      self.batch_size = batch_size
      
      self.enc = encoder.get_encoder(model_name)
      hparams = model.default_hparams()
      with open(os.path.join('models', model_name, 'hparams.json')) as f:
          hparams.override_from_dict(json.load(f))

      if length is None:
          length = hparams.n_ctx // 2
      elif length > hparams.n_ctx:
          raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

      self.sess = tf.Session(graph=tf.Graph())
      self.sess.__enter__()
      
      self.context = tf.placeholder(tf.int32, [batch_size, None])
      np.random.seed(seed)
      tf.set_random_seed(seed)
      self.output = sample.sample_sequence(
          hparams=hparams, length=length,
          context=self.context,
          batch_size=batch_size,
          temperature=temperature, top_k=top_k
      )

      saver = tf.train.Saver()
      self.ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
      saver.restore(self.sess, self.ckpt)

  def close(self):
    self.sess.close()
  
  def generate_conditional(self,raw_text):
      context_tokens = self.enc.encode(raw_text)
      generated = 0
      for _ in range(self.nsamples // self.batch_size):
          out = self.sess.run(self.output, feed_dict={
              self.context: [context_tokens for _ in range(self.batch_size)]
          })[:, len(context_tokens):]
          for i in range(self.batch_size):
              generated += 1
              text = self.enc.decode(out[i])
              return text
              #print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
              #print(text)
      #print("=" * 80)

In [ ]:
# Carregar o modelo
gpt2 = GPT2(model_name=chatbot_model)

In [ ]:
result = gpt2.generate_conditional(raw_text="How are you?")

print(result)

# **Troubleshooting**
No caso de problemas, mate o ambiente e recrie-o. O comando abaixo faz isso. Lembre-se: ao redefinir o ambiente, será necessário baixar o modelo, iniciar o TensorFlow e baixar todos os seus arquivos de volta do drive.

In [ ]:
# Reiniciar o ambiente
!kill -9 -1